In [ ]:
import pandas as pd
import numpy as np

abm = pd.read_csv("abm.csv")
card = pd.read_csv("card.csv")
cheque = pd.read_csv("cheque.csv")
eft = pd.read_csv("eft.csv")
emt = pd.read_csv("emt.csv")
kyc = pd.read_csv("kyc.csv")
kyc_industry_codes = pd.read_csv("kyc_industry_codes.csv")
wire = pd.read_csv("wire.csv")

emt['debit_credit'] = emt['debit_credit'].map({'C': 'credit', 'D': 'debit'})

# Add transaction type and ID columns
abm['transaction_type'] = 'ABM'
abm['transaction_id'] = abm['abm_id']
card['transaction_type'] = 'Card'
card['transaction_id'] = card['card_trxn_id']
cheque['transaction_type'] = 'Cheque'
cheque['transaction_id'] = cheque['cheque_id']
eft['transaction_type'] = 'EFT'
eft['transaction_id'] = eft['eft_id']
emt['transaction_type'] = 'EMT'
emt['transaction_id'] = emt['emt_id']
wire['transaction_type'] = 'Wire'
wire['transaction_id'] = wire['wire_id']

# Standardize column names and select relevant columns
abm = abm[['transaction_id', 'customer_id', 'amount_cad', 'debit_credit', 'transaction_date', 'transaction_time', 'transaction_type']]
card = card[['transaction_id', 'customer_id', 'amount_cad', 'debit_credit', 'transaction_date', 'transaction_time', 'transaction_type']]
cheque = cheque[['transaction_id', 'customer_id', 'amount_cad', 'debit_credit', 'transaction_date', 'transaction_type']]
eft = eft[['transaction_id', 'customer_id', 'amount_cad', 'debit_credit', 'transaction_date', 'transaction_time', 'transaction_type']]
emt = emt[['transaction_id', 'customer_id', 'amount_cad', 'debit_credit', 'transaction_date', 'transaction_time', 'transaction_type']]
wire = wire[['transaction_id', 'customer_id', 'amount_cad', 'debit_credit', 'transaction_date', 'transaction_time', 'transaction_type']]

transactions = pd.concat([abm, card, cheque, eft, emt, wire], ignore_index=True)
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'])
transactions['transaction_datetime'] = transactions.apply(lambda row: pd.to_datetime(row['transaction_date'].strftime('%Y-%m-%d') + ' ' + (row['transaction_time'] if isinstance(row.get('transaction_time'), str) else '00:00:00')), axis=1)
transactions = transactions.drop(columns=['transaction_date', 'transaction_time'])

# Merge with KYC data
merged_data = pd.merge(transactions, kyc, on='customer_id', how='left')

# Reorder columns
merged_data = merged_data[['transaction_id', 'customer_id', 'amount_cad', 'debit_credit', 'transaction_datetime', 'transaction_type', 'country', 'province', 'city', 'industry_code', 'employee_count', 'sales', 'established_date', 'onboard_date']]


print(merged_data.head())
print(merged_data.info())

merged_data.to_csv("merged_data.csv", index=False)

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Industry average for each transaction type, diff between the average and the transaction amount
# 